In [8]:
import time
import numpy as np

from matplotlib import pyplot, patches

from sklearn.datasets import make_classification, load_breast_cancer, load_iris
from sklearn.preprocessing import minmax_scale, normalize
from sklearn.discriminant_analysis import LinearDiscriminantAnalysis
from sklearn.linear_model import LogisticRegression, Perceptron
from sklearn.model_selection import KFold, LeaveOneOut


from scipy.stats import norm

from sklearn.neighbors import KernelDensity

In [9]:
class DiscriminantLineaire:
    def __init__(self, eta=1e-2, epsilon=1e-3, max_iter=1000):
        # Cette fonction est déjà  codée pour vous, vous n'avez qu'à  utiliser
        # les variables membres qu'elle définit dans les autres fonctions de
        # cette classe.
        self.eta = eta
        self.epsilon = epsilon
        self.max_iter = max_iter

    def fit(self, X, y):
        if set(y) == {0, 1}:
            y = (y * 2) - 1
        elif set(y) != {-1, 1}:
            raise Exception("The target must be {0,1} or {-1,1}")
        # Implémentez la fonction d'entraînement du classifieur, selon
        # les équations que vous avez développées dans votre rapport.

        # On initialise les poids aléatoirement
        w = np.random.rand(X.shape[1] + 1)
        # Je prend w[0] = w0 et w[1:] tout le reste des poids
        # TODO Q2B
        # Vous devez ici implémenter l'entraînement.
        # Celui-ci devrait être contenu dans la boucle suivante, qui se répète
        # self.max_iter fois
        # Vous êtes libres d'utiliser les noms de variable de votre choix, sauf
        # pour les poids qui doivent être contenus dans la variable w définie
        # plus haut
        Err_prec = 1e10
        for i in range(self.max_iter):
            print("boucle : ", i)
            num_mal_classe = []
            ind_mal_classe = []

            for j in range(len(y)):
                hxjw = np.dot(w[1:], X[j]) + w[0]
                Y = y[j] * hxjw
                if Y <= 0:
                    num_mal_classe.append(y[j] - hxjw)
                    ind_mal_classe.append(j)
            ind_mal_classe = np.array(ind_mal_classe)
            if ind_mal_classe.size:
                normeX = np.linalg.norm(X[ind_mal_classe])**2
                num = np.array(num_mal_classe)
                Err = np.array(0.5 * sum(num**2 / normeX))
                deltawi = self.eta * \
                    [sum(np.dot(X[ind_mal_classe][:, i], (num / (normeX))))
                     for i in range()]
                deltaw0 = self.eta * sum(num / normeX)
            else:
                Err = 0
                deltawi = 0
                deltaw0 = 0

            # condition d'arrêt :
            print("Err : ", Err)
            if Err_prec - Err < self.epsilon:
                print("Les poids ont convergés")
                break
            else:
                Err_prec = Err
            print("dela : ", deltawi)
            w[1:] += deltawi
            w[0] += deltaw0

            # à ce stade, la variable w devrait contenir les poids entraînés
            # On les copie dans une variable membre pour les conserver
            print("poids : ", w[1:])
            print("Base : ", w[0])
            print()
        self.w = w

    def predict(self, X):
        # TODO Q2B
        # Implémentez la fonction de prédiction
        # Vous pouvez supposer que fit() a préalablement été exécuté
        return np.array(
            [1 if self.w[1:].dot(x) + self.w[0] >= 0
             else 0
             for x in X])

    def score(self, X, y):
        if set(y) == {0, 1}:
            y = (y * 2) - 1
        elif set(y) != {-1, 1}:
            raise Exception("The target must be {0,1} or {-1,1}")
        # TODO Q2B
        # Implémentez la fonction retournant le score (précision / accuracy)
        # du classifieur sur les données reçues en argument.
        # Vous pouvez supposer que fit() a préalablement été exécuté
        # Indice : réutiliser votre implémentation de predict() réduit de
        # beaucoup la taille de cette fonction!

        # Question 2B
        # Implémentation du classifieur un contre tous utilisant le
        # discriminant linéaire défini plus haut
        return sum([self.predict(X)[i] == y[i]
                    for i in range(len(y))]) / len(y)


In [10]:
class ClassifieurUnContreTous:
    def __init__(self, n_classes, **kwargs):
        # Cette fonction est déjà  codée pour vous, vous n'avez qu'à  utiliser
        # les variables membres qu'elle définit dans les autres fonctions de
        # cette classe.
        self.n_classes = n_classes
        self.estimators = [DiscriminantLineaire(
            **kwargs) for c in range(n_classes)]

    def fit(self, X, y):
        # TODO Q2C
        # Implémentez ici une approche un contre tous, oà¹ chaque classifieur
        # (contenu dans self.estimators) est entraîné à  distinguer une seule
        # classe versus toutes les autres
        for i in range(self.n_classes):
            target = y - i
            target[numpy.where(target != 0)] = 1
            # target = (target*2)-1 pour mettre entre -1 et 1
            # mais déjà implémenté dans le fit de Disciminant linéraire
            self.estimators[i].fit(X, target)

    def predict(self, X):
        # TODO Q2C
        # Implémentez ici la prédiction utilisant l'approche un contre tous
        # Vous pouvez supposer que fit() a préalablement été exécuté
        classes = []
        for x in X:
            Hall = [est.w[1:].dot(x) + est.w[0] for est in self.estimators]
            classes.append(numpy.argmax(Hall))
        return classes

    def score(self, X, y):
        # TODO Q2C
        # Implémentez ici le calcul du score utilisant l'approche un contre
        # tous. Ce score correspond à  la précision (accuracy) moyenne.
        # Vous pouvez supposer que fit() a préalablement été exécuté
        return sum([self.predict(X)[i] == y[i] for i in range(len(y))])/len(y)

In [11]:
X, y = make_classification(n_features=2, n_redundant=0, n_informative=2,
                           n_clusters_per_class=1)


clf = DiscriminantLineaire(epsilon=1e-2)
clf.fit(X, y)
fig, sfig = pyplot.subplots(1, 1, sharex=True, sharey=True)

pas = 100
x1 = numpy.linspace(X[:, 0].min(), X[:, 0].max(), pas)
x2 = numpy.linspace(X[:, 1].min(), X[:, 1].max(), pas)
absc, ordon = numpy.meshgrid(x1, x2)
sfig.scatter(absc, ordon, alpha=0.05, s=20,
             c=["bgrcmykw"[i] for i in clf.predict(
                 numpy.c_[absc.ravel(), ordon.ravel()])])
sfig.scatter(X[:, 0], X[:, 1], c=["bgrcmykw"[i] for i in y])

pyplot.show()

boucle :  0


TypeError: range expected 1 arguments, got 0

In [14]:
w = np.random.rand(X.shape[1] + 1)
num_mal_classe = []
ind_mal_classe = []
for j in range(len(y)):
    hxjw = np.dot(w[1:], X[j]) + w[0]
    Y = y[j] * hxjw
    if Y <= 0:
        num_mal_classe.append(y[j] - hxjw)
        ind_mal_classe.append(j)
ind_mal_classe = np.array(ind_mal_classe)

In [20]:
normeX = np.linalg.norm(X[ind_mal_classe])**2
num = np.array(num_mal_classe)
Err = np.array(0.5 * sum(num**2 / normeX))
deltawi = 0.45 * \
    [sum(np.dot(X[ind_mal_classe][:, i], (num / (normeX))))
     for i in range(X.shape[1])]

TypeError: 'numpy.float64' object is not iterable

In [31]:
list(range(X.shape[1]))

[0, 1]

array([ 0.2452442 , -1.09082819, -2.48536686, -0.98027382, -1.46038448,
        0.08652863, -2.47693199, -0.47956659,  0.11278721, -1.31546898,
       -1.56306676, -0.35289364, -1.59552161, -2.08550484, -2.49207855,
        0.15685036,  0.50986526, -2.97213229,  0.13892327, -0.46901274,
       -2.12840186, -0.63896116,  0.52051325, -0.63171566, -0.98153321,
       -2.65858061, -1.11961035, -1.16722561, -0.19125828, -1.71290441,
       -1.38160063, -1.43662287, -1.58684661, -1.23260598, -1.08206529,
       -1.9786807 , -1.48209328,  0.0162584 , -0.73971063, -2.2988313 ,
       -1.94156856,  0.58365125, -2.78451936, -0.65602586, -0.1545911 ,
       -1.19867626, -1.30801144,  0.2437647 ,  0.75680413,  0.7467427 ,
        0.21206046, -0.92779235, -2.0803194 , -0.46411589,  1.22090053,
       -0.03679753, -0.78479731, -0.51251437, -1.36260571, -1.79359204,
       -1.42532252, -1.23197245, -2.79061175, -0.23247328,  0.02102749])

In [42]:
plop = [np.dot(X[ind_mal_classe][:,i],(num/normeX)) for i in range(X.shape[1])]

In [43]:
plop


[-0.11834317217656982, -0.26674875957511435]

In [45]:
plop = np.array(plop)

In [46]:
plop

array([-0.11834317, -0.26674876])

In [47]:
plop *= 0.4

In [48]:
plop

array([-0.04733727, -0.1066995 ])

In [54]:
w = np.array([-0.0069122 , -0.00353473])
delta = np.array([0.2535428  ,0.85162685])


In [55]:
w += delta

In [58]:
X= load_breast_cancer()

In [70]:
minmax_scale(X.data,feature_range=(0, 1))



array([[0.52103744, 0.0226581 , 0.54598853, ..., 0.91202749, 0.59846245,
        0.41886396],
       [0.64314449, 0.27257355, 0.61578329, ..., 0.63917526, 0.23358959,
        0.22287813],
       [0.60149557, 0.3902604 , 0.59574321, ..., 0.83505155, 0.40370589,
        0.21343303],
       ...,
       [0.45525108, 0.62123774, 0.44578813, ..., 0.48728522, 0.12872068,
        0.1519087 ],
       [0.64456434, 0.66351031, 0.66553797, ..., 0.91065292, 0.49714173,
        0.45231536],
       [0.03686876, 0.50152181, 0.02853984, ..., 0.        , 0.25744136,
        0.10068215]])

In [69]:
normalize(X.data)

array([[7.92541486e-03, 4.57286305e-03, 5.40989964e-02, ...,
        1.16920795e-04, 2.02695018e-04, 5.23808686e-05],
       [8.66575595e-03, 7.48616836e-03, 5.59882822e-02, ...,
        7.83583182e-05, 1.15852352e-04, 3.75024596e-05],
       [9.36668268e-03, 1.01087865e-02, 6.18419883e-02, ...,
        1.15596947e-04, 1.71873157e-04, 4.16624718e-05],
       ...,
       [1.16438847e-02, 1.96964025e-02, 7.59658259e-02, ...,
        9.94640269e-05, 1.55579134e-04, 5.48525170e-05],
       [9.23020523e-03, 1.31418408e-02, 6.27743569e-02, ...,
        1.18738077e-04, 1.83125480e-04, 5.55604587e-05],
       [2.31098880e-02, 7.30820427e-02, 1.42709515e-01, ...,
        0.00000000e+00, 8.55006294e-04, 2.09626935e-04]])

In [82]:
for i in range(4):
    if bin(i)[-2] == 'b':
        nb_subplot = int('0'),int(bin(i)[-1])
    else:
        nb_subplot = int(bin(i)[-2]),int(bin(i)[-1])
    print(nb_subplot)

(0, 0)
(0, 1)
(1, 0)
(1, 1)


In [105]:
data = load_iris()
X, y = minmax_scale(data.data, feature_range=(0, 1)), data.target

# TODO Q2D
# Comparez les diverses approches demandées dans l'énoncé sur Iris
# Pour utilisez votre discriminant linéaire, utilisez l'approche Un Contre
# Tous implémenté au 2C.
# Initialisez vos discriminants linéaires avec les paramètres suivants :
# DiscriminantLineaire(eta=1e-4, epsilon=1e-6, max_iter=10000)
# Pour les autres approches, conservez les valeurs par défaut
# N'oubliez pas que l'évaluation doit être faite par une validation
# croisée à  K=3 plis!


In [86]:
clfs = [ClassifieurUnContreTous(len(X[0]), eta=1e-4,
                                epsilon=1e-6, max_iter=10000),
        LinearDiscriminantAnalysis(),
        Perceptron(),
        LogisticRegression()]

In [87]:
kf = KFold(n_splits=3, random_state=666, shuffle=False)
ErrorsTest = [0] * 4
ErrorsTrain = [0] * 4

In [90]:
for i in range(len(clfs)):
    avgErrorTest = 0
    avgErrorTrain = 0
    for train_index, test_index in kf.split(X):
        X_train, X_test = X[train_index], X[test_index]
        y_train, y_test = y[train_index], y[test_index]
        print(y_train)

[1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1
 1 1 1 1 1 1 1 1 1 1 1 1 1 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2
 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2]
[0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0
 0 0 0 0 0 0 0 0 0 0 0 0 0 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2
 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2]
[0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0
 0 0 0 0 0 0 0 0 0 0 0 0 0 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1
 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1]
[1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1
 1 1 1 1 1 1 1 1 1 1 1 1 1 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2
 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2]
[0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0
 0 0 0 0 0 0 0 0 0 0 0 0 0 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2
 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2

In [92]:
for i in set([0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2]):
    target = y - i
    target[np.where(target != 0)] = 1
    print(target)

[0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0
 0 0 0 0 0 0 0 0 0 0 0 0 0 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1
 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1
 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1
 1 1]
[1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1
 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1
 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 0 0 0 0 0 0 0 0 0 0 0
 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0
 0 0]


In [106]:
y

array([0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 0, 0, 0, 0, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1,
       1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1,
       1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2,
       2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2,
       2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2])

In [109]:
for i in range(3):
    target = y.copy()
    target[np.where(target == i)] = 0
    target[np.where(target != 0)] = 1
    print(target)

[0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0
 0 0 0 0 0 0 0 0 0 0 0 0 0 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1
 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1
 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1
 1 1]
[0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0
 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0
 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 1 1 1 1 1 1 1 1 1 1 1
 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1
 1 1]
[0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0
 0 0 0 0 0 0 0 0 0 0 0 0 0 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1
 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 0 0 0 0 0 0 0 0 0 0 0
 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0
 0 0]


In [108]:
target

array([0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 0, 0, 0, 0, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1,
       1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1,
       1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0])

In [85]:



for i in range(len(clfs)):
    avgErrorTest = 0
    avgErrorTrain = 0
    for train_index, test_index in kf.split(X):
        X_train, X_test = X[train_index], X[test_index]
        y_train, y_test = y[train_index]-1, y[test_index]-1
        clfs[i].fit(X_train, y_train)
        avgErrorTrain += 1 - clfs[i].score(X_train, y_train)
        avgErrorTest += 1 - clfs[i].score(X_test, y_test)
    ErrorsTest[i] = avgErrorTest / 3
    ErrorsTrain[i] = avgErrorTrain / 3

print("Erreurs Iris Train (Own, Linear, Perceptron, Logistic) :\n",
      ErrorsTrain)
print("Erreurs Iris Test (Own, Linear, Perceptron, Logistic) :\n",
      ErrorsTest)


boucle :  0


TypeError: range expected 1 arguments, got 0

In [115]:
for i,j in zip([1,2,3],[18,19,20]):
    print(i,j)

1 18
2 19
3 20
